In [14]:
import pandas as pd

from algos import MySVD, MyPLSA, LDAAdapter
from metrics import mean_average_precision
from splitter import split
from utils import Transformer, create_negative_examples

In [8]:
random_state = 50

In [9]:
df = pd.read_csv('data/small_data.csv')

In [10]:
df_train, df_test = split(df, test_size=0.5, random_state=random_state)

In [11]:
df_train.head()

,group,user
21295,64180326,193032199
25417,48329519,115971270
132786,60958801,132225098
63930,89385533,6349270
29875,40207499,171585340


In [27]:
# подобрано в hyper_opt.ipynb
svd_argmax_combo = {'num_topics': 3}

In [28]:
# SVD
df_neg = create_negative_examples(df=df, df_train=df_train,
                                  num_neg_exs=10*df_train.shape[0], random_state=random_state)
svd = MySVD()
svd.set_params(**svd_argmax_combo)

svd.fit(df_train, df_neg)

# transform df_test
svd_group_encoder_dict = dict(zip(svd.group_encoder.classes_, list(range(len(svd.group_encoder.classes_)))))
encoded_corpus_test = df_test.groupby("user").agg(lambda x: [svd_group_encoder_dict[item] for item in x]).reset_index()

encoded_corpus_test['recs'] = svd.predict(encoded_corpus_test.user.values)
print('SVD result: {}'.format(mean_average_precision(encoded_corpus_test.group.values, encoded_corpus_test.recs.values)))

SVD result: 0.17991243739157747


In [21]:
# подобрано в hyper_opt.ipynb
plsa_argmax_combo = {'num_topics': 3}

In [29]:
# PLSA
transformer = Transformer()
transformer.fit(df)

corpus_train = transformer.transform(df_train)
corpus_test = transformer.transform(df_test)


est = MyPLSA()
est.set_params(**plsa_argmax_combo)

num_groups = df.group.nunique()
est.fit(corpus_train, num_groups)

corpus_test['recs'] = est.predict(corpus_test.user.values)

corpus_test['group'] = corpus_test.group.apply(lambda x: [item[0] for item in x])
print('PLSA result: {}'.format(mean_average_precision(corpus_test.group.values, corpus_test.recs.values)))

PLSA result: 0.151125658012547


In [23]:
# подобрано в hyper_opt.ipynb
lda_argmax_combo = {'num_topics': 3}

In [30]:
# LDA
transformer = Transformer()

transformer.fit(df)

corpus_train = transformer.transform(df_train)
corpus_test = transformer.transform(df_test)


est = LDAAdapter()
est.set_params(**lda_argmax_combo)

est.fit(corpus_train)

corpus_test['recs'] = est.predict(corpus_test.user.values)

corpus_test['group'] = corpus_test.group.apply(lambda x: [item[0] for item in x])
print('LDA result: {}'.format(mean_average_precision(corpus_test.group.values, corpus_test.recs.values)))

LDA result: 0.41391515219393127


ВЫВОД:Лучше всех качество по метрике mean average precision даёт алгоритм, основанный на LDA. Возможно, это связано с тем, что другие алгоритмы я реализовывал сам и для ускорения обнаружения сходимости итеративных методов использовал не полное вычисление нормы Фробениуса матриц, а приближенные аналоги показателей сходимости (когда повысил точность идентификации сходимости в SVD, качество значительно выросло).